# Preference Alignment with GRPO

- https://huggingface.co/docs/trl/main/en/grpo_trainer#trl.GRPOTrainer

## Import libraries


In [1]:
!uv pip install transformers datasets trl huggingface_hub --system

Using Python 3.11.11 environment at /usr
Resolved 61 packages in 498ms
⠙ Preparing packages... (0/16)
⠙ Preparing packages... (0/16)
⠙ Preparing packages... (0/16)
nvidia-cusolver-cu12 ------------------------------ 16.00 KiB/122.01 MiB
⠙ Preparing packages... (0/16)
nvidia-cusolver-cu12 ------------------------------ 16.00 KiB/122.01 MiB
⠙ Preparing packages... (0/16)
nvidia-cuda-nvrtc-cu12 ------------------------------     0 B/23.50 MiB
nvidia-cusolver-cu12 ------------------------------ 16.00 KiB/122.01 MiB
⠙ Preparing packages... (0/16)
nvidia-cuda-nvrtc-cu12 ------------------------------ 14.89 KiB/23.50 MiB
nvidia-cusolver-cu12 ------------------------------ 16.00 KiB/122.01 MiB
⠙ Preparing packages... (0/16)
nvidia-cuda-cupti-cu12 ------------------------------     0 B/13.17 MiB
nvidia-cuda-nvrtc-cu12 ------------------------------ 14.89 KiB/23.50 MiB
nvidia-cusolver-cu12 ------------------------------ 16.00 KiB/122.01 MiB
⠙ Preparing packages... (0/16)
nvidia-cuda-cupti-cu12 -

In [2]:
import torch
import os
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)
from trl import GRPOConfig, GRPOTrainer

In [3]:
import os
from getpass import getpass
os.environ['HF_TOKEN'] = getpass()

··········


In [4]:
dataset = load_dataset("maywell/ko_Ultrafeedback_binarized", token=os.environ['HF_TOKEN'])

README.md:   0%|          | 0.00/819 [00:00<?, ?B/s]

(…)-00000-of-00001-dc7eba5173eb6ca1.parquet:   0%|          | 0.00/110M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/61966 [00:00<?, ? examples/s]

## Format dataset

In [5]:
def process_dataset(data):
    return {"prompt": [{"role": "user", "content": data["prompt"]}]}

train_ds = dataset.map(process_dataset)

Map:   0%|          | 0/61966 [00:00<?, ? examples/s]

## Define the model

In [7]:
model_name = "josang1204/Qweb2.5-FT-CSY"

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Model to fine-tune
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name,
    torch_dtype=torch.float32,
    token=os.environ['HF_TOKEN']
).to(device)
model.config.use_cache = False
tokenizer = AutoTokenizer.from_pretrained(model_name, token=os.environ['HF_TOKEN'])

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.26k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

In [8]:
finetune_name = "Qweb2.5-FT-GRPO-CSY"
finetune_tags = ["smol-course", "module_2"]

## Train model with GRPO

In [9]:
grpo_args = GRPOConfig(
    logging_steps=10,
    output_dir="./results/",
    hub_model_id=finetune_name,
)

In [10]:
def reward_len(completions, **kwargs):
    return [abs(20 - len(completion)) for completion in completions]

trainer = GRPOTrainer(
    model=model,
    args=grpo_args,
    reward_funcs=reward_len,
    train_dataset=dataset["train"],
    processing_class=tokenizer,
)

In [ ]:
trainer.train()  # Train the model

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: csy1204 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss


In [ ]:
trainer.save_model(f"./{finetune_name}")

In [ ]:
trainer.push_to_hub(tags=finetune_tags, token=os.environ["HF_TOKEN"])

## 💐 You're done!

This notebook provided a step-by-step guide to fine-tuning the `HuggingFaceTB/SmolLM2-135M` model using the `ORPOTrainer`. By following these steps, you can adapt the model to perform specific tasks more effectively. If you want to carry on working on this course, here are steps you could try out:

- Try this notebook on a harder difficulty
- Review a colleagues PR
- Improve the course material via an Issue or PR.